## Задача №1

Посчитайте, сколько компаний закрылось.

In [ ]:
SELECT COUNT (status)
FROM company 
WHERE status = 'closed';

## Задача №2

Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total .

In [ ]:
SELECT funding_total
FROM company
WHERE category_code = 'news' AND country_code = 'USA'
ORDER BY funding_total DESC

## Задача №3

Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash' and acquired_at between '2011-01-01' and '2013-12-31'

## Задача №4

Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.

In [ ]:
SELECT first_name,
       last_name,
       twitter_username
FROM  people
WHERE twitter_username LIKE 'Silver%'

## Задача №5

Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'.

In [ ]:
SELECT *
FROM people 
WHERE twitter_username LIKE '%money%' AND last_name LIKE 'K%'

## Задача №6

Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:
SELECT country_code,
       SUM(funding_total) as sum
FROM company
GROUP BY country_code
ORDER BY sum DESC

## Задача №7

Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.

Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
SELECT funded_at,
       MIN(raised_amount),
       MAX(raised_amount)
FROM funding_round
group by funded_at
HAVING MIN(raised_amount) != 0 and MIN(raised_amount) != MAX (raised_amount);

## Задача №8

Создайте поле с категориями:
- Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
- Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
- Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.

Отобразите все поля таблицы fund и новое поле с категориями.

In [ ]:
SELECT *,
    CASE 
        WHEN invested_companies >= 100 THEN 'high_activity'
        WHEN invested_companies >= 20 and invested_companies < 100 THEN 'middle_activity'
        ELSE 'low_activity'
     END 
FROM fund 

## Задача №9

Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
SELECT ROUND(AVG(investment_rounds)),
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity
FROM fund
GROUP BY activity
ORDER BY ROUND(AVG(investment_rounds))

## Задача №10

Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 

Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. Выгрузите десять самых активных стран-инвесторов.

Отсортируйте таблицу по среднему количеству компаний от большего к меньшему, а затем по коду страны в лексикографическом порядке.

In [ ]:
SELECT country_code,
       MIN (invested_companies),
       MAX(invested_companies),
       AVG(invested_companies)
FROM fund
WHERE founded_at BETWEEN '2010-01-01' AND '2012-12-31'
GROUP BY country_code
HAVING MIN(invested_companies) != 0
ORDER BY AVG (invested_companies) DESC, country_code
LIMIT 10

## Задача №11

Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
SELECT p.first_name,
       p.last_name,
       e.instituition
FROM people as p
LEFT JOIN education as e on p.id = e.person_id

## Задача №12

Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [ ]:
SELECT c.name,
      COUNT (DISTINCT e.instituition)
FROM company as c

JOIN people as p ON c.id=p.company_id
JOIN education as e ON p.id=e.person_id

GROUP BY c.name 
ORDER BY COUNT(DISTINCT e.instituition) DESC
LIMIT 5;

## Задача №13

Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
SELECT DISTINCT c.name
FROM company as c
WHERE id IN (SELECT company_id
            FROM funding_round
            WHERE is_first_round = 1 AND is_last_round = 1)
AND status = 'closed'



## Задача №14

Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
SELECT DISTINCT p.id
FROM people as p
WHERE company_id in (SELECT id
                    FROM company as c
                    WHERE id IN (SELECT company_id
                                 FROM funding_round
                                 WHERE is_first_round = 1 AND is_last_round = 1)
                    AND status = 'closed')



## Задача №15


Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
SELECT DISTINCT p.id,
       e.instituition
       
FROM (SELECT id  
      FROM people as p
      WHERE company_id in (SELECT id
                    FROM company as c
                    WHERE id IN (SELECT company_id
                                 FROM funding_round
                                 WHERE is_first_round = 1 AND is_last_round = 1)
                    AND status = 'closed')) as p
                    
 JOIN

( SELECT person_id,
         instituition
    FROM education) as e ON p.id=e.person_id


## Задача №16

Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [ ]:
SELECT DISTINCT p.id as id,
      COUNT(e.instituition) as instituition
       
FROM (SELECT id  
      FROM people as p
      WHERE company_id in (SELECT id
                    FROM company as c
                    WHERE id IN (SELECT company_id
                                 FROM funding_round
                                 WHERE is_first_round = 1 AND is_last_round = 1)
                    AND status = 'closed')) as p
                    
 JOIN

( SELECT person_id,
         instituition
    FROM education) as e ON p.id=e.person_id

GROUP BY id


## Задача №17

Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
SELECT SUM(e.instituition)/COUNT(e.id)
FROM
(SELECT DISTINCT p.id as id,
       COUNT(e.instituition) as instituition
       
FROM (SELECT id  
      FROM people as p
      WHERE company_id in (SELECT id
                    FROM company as c
                    WHERE id IN (SELECT company_id
                                 FROM funding_round
                                 WHERE is_first_round = 1 AND is_last_round = 1)
                    AND status = 'closed')) as p
                    
 JOIN

( SELECT person_id,
         instituition
    FROM education) as e ON p.id=e.person_id

GROUP BY id) as e


## Задача №18

Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook*.
*(сервис, запрещённый на территории РФ)

In [ ]:
SELECT SUM(e.instituition)/COUNT(e.id)
FROM
(SELECT DISTINCT p.id as id,
       COUNT(e.instituition) as instituition
       
FROM (SELECT id  
      FROM people as p
      WHERE company_id in (SELECT id
                    FROM company as c
                    WHERE name = 'Facebook')) as p
                    
 JOIN

( SELECT person_id,
         instituition
    FROM education) as e ON p.id=e.person_id

GROUP BY id) as e


## Задача №19


Составьте таблицу из полей:
- name_of_fund — название фонда;
- name_of_company — название компании;
- amount — сумма инвестиций, которую привлекла компания в раунде.

В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
select f.name as name_of_fund,
       c.name as name_of_company,
       fr.raised_amount as amount
from investment as i
    join company as c ON i.company_id=c.id 
    join fund as f ON i.fund_id=f.id 
    join funding_round as fr ON i.funding_round_id=fr.id 
where с.milestones > 6 and
            extract(year from cast(fr.funded_at as date)) between '2012' and '2013' 


## Задача №20


Выгрузите таблицу, в которой будут такие поля:
- название компании-покупателя;
- сумма сделки;
- название компании, которую купили;
- сумма инвестиций, вложенных в купленную компанию;
- доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.

Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 

Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
SELECT buy.name,
       a.price_amount,
       sell.name,
       sell.funding_total,
       ROUND(a.price_amount/sell.funding_total)

FROM acquisition AS a
LEFT JOIN company AS buy ON a.acquiring_company_id=buy.id
LEFT JOIN company AS sell ON a.acquired_company_id=sell.id

WHERE a.price_amount !=0
AND sell.funding_total !=0
ORDER BY a.price_amount DESC, sell.name 
LIMIT 10;

## Задача №21

Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
SELECT c.name as name,
       EXTRACT (MONTH FROM CAST(fr.funded_at as date))
FROM company as c
    LEFT JOIN funding_round as fr ON c.id = fr.company_id
WHERE category_code = 'social'
       AND fr.funded_at BETWEEN '2010-01-01' AND '2013-12-31' 
       AND fr.raised_amount != 0

## Задача №22

Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
- номер месяца, в котором проходили раунды;
- количество уникальных названий фондов из США, которые инвестировали в этом месяце;
- количество компаний, купленных за этот месяц;
- общая сумма сделок по покупкам в этом месяце.

In [ ]:
WITH
uno AS (SELECT COUNT(DISTINCT f.name),
               EXTRACT(MONTH FROM CAST(fr.funded_at AS DATE))
        FROM funding_round AS fr
        LEFT JOIN investment AS i ON fr.id=i.funding_round_id
        LEFT JOIN fund AS f ON i.fund_id=f.id
        WHERE EXTRACT(YEAR FROM CAST(fr.funded_at AS DATE)) BETWEEN 2010 AND 2013
        AND f.country_code = 'USA'
        GROUP BY EXTRACT(MONTH FROM CAST(fr.funded_at AS DATE))),
        
due AS (SELECT EXTRACT(MONTH FROM CAST(a.acquired_at AS DATE)),
                COUNT(a.acquired_company_id),
                SUM(a.price_amount)
        FROM acquisition AS a
        WHERE EXTRACT(YEAR FROM CAST(a.acquired_at AS DATE)) BETWEEN 2010 AND 2013
        GROUP BY EXTRACT(MONTH FROM CAST(a.acquired_at AS DATE)))
        
SELECT uno.date_part AS month,
        uno.count AS funds,
        due.count AS startups,
        due.sum
FROM uno
FULL OUTER JOIN due ON uno.date_part=due.date_part;

## Задача №23

Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
SELECT A.country_code, A.year_2011, B.year_2012, C.year_2013
from
(select avg(funding_total) as year_2011,
    country_code
    from company
    where extract(year from cast(founded_at as date))=2011
group by country_code) as A

INNER JOIN

(select avg(funding_total) as year_2012,
    country_code
    from company
    where extract(year from cast(founded_at as date))=2012
group by country_code) as B on
A.country_code=B.country_code

INNER JOIN 

(select avg(funding_total) as year_2013,
    country_code
    from company
    where extract(year from cast(founded_at as date))=2013
group by country_code) as C on 
A.country_code=C.country_code


order by A.year_2011 desc;